In [1]:
# default_exp data

# Setup functions

In [2]:
#hide
from nbdev.showdoc import *

# Setup functions
> get_kaggle_dataset(competition)
> pathFromCompetition(competition)
> download_and_log(competition)
> setup(competition)

In [3]:
# !pip install -Uqq kaggle
# !pip install -Uqq wandb
# !pip install -Uqq python-dotenv
!pip install -Uqq pytest

In [19]:
# export
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
# https://technowhisp.com/kaggle-api-python-documentation/
from kaggle.api.kaggle_api_extended import KaggleApi
from fastai.vision.all import *
import wandb
import os
from tqdm import tqdm

In [5]:
load_dotenv()
k = KaggleApi()
k.authenticate()

## Download data from Kaggle
> Download any competition dataset and extract to expected fastai folder.  Uses fastai config to find out where to extract the dataset locally.

In [6]:
#export 
def get_kaggle_dataset(competition):
    """ Download and extract kaggle competition dataset to fastai data folder and return path to files.
    get_kaggle_dataset(string) -> Path
    args:
        competition: string containing competition name in Kaggle
    returns:
        path to local competition data"""
    config = Config()
    zipdest = Path(config.d['archive_path'])
    zipname = Path(config.d['archive_path']+'/'+competition+'.zip')
    if not zipname.exists():
        print (f'${zipname} does not exist.')
        try:
            k.competition_download_files(competition, path=zipdest)
        except:
            raise Exception(f'No competition {competition} found at Kaggle.')
    print('Downloading....')
    dest = Path(config.d['data_path']+'/'+competition+'/')
    if not dest.exists():
        print (f'${dest} does not exist.  Extracting...')
        file_extract(zipname, dest=dest)
        files = os.listdir(dest)
        for f in files:
            if f.split('.')[-1]=='zip':
                print(dest/f)
                file_extract(dest/f)
                os.remove(dest/f)
    return dest

In [7]:
import pytest
with pytest.raises(Exception) as e_info:
    get_kaggle_dataset('blah')
assert "Exception" in str(e_info)

$/home/fred/.fastai/archive/blah.zip does not exist.


In [8]:
k.competitions_list(search='siim')

[siim-covid19-detection,
 siim-isic-melanoma-classification,
 siim-acr-pneumothorax-segmentation]

In [9]:
#p = get_kaggle_dataset('dogs-vs-cats-redux-kernels-edition')
#p

In [10]:
# export
def pathFromCompetition(competition):
    """Returns path to local competition files.
    args:
    competition: string containing name of competition"""
    config = Config()
    ret =  Path(config.d['data_path']+'/'+competition+'/')
    if not ret.exists(): raise Exception('Please download the competition data first.')
    return ret

In [11]:
# export
def confirm(msg=""):
    """
    Ask user to enter Y or N (case-insensitive).
    :return: True if the answer is Y.
    :rtype: bool
    """
    answer = ""
    if not msg: msg = "OK to continue"
    while answer not in ["y", "n"]:
        answer = input(msg+" [Y/N]? ").lower()
    return answer == "y"

## Download data from Kaggle and log at WandB


In [12]:
# export
def download_and_log(competition, entity=None):
    """ Start a Run for data download at WandB, download competition data and log data reference in local machine as an Artifact at WandB.
    args:
        competition: string containing competition name
    returns:
        Path to competition data
    """
    try:
        p = pathFromCompetition(competition)
        if not confirm(f'Data for competition found at {p}\n Force new download '): return p
    except: pass
    try:
        k.competition_list_files(competition)
    except:
        print (f'No competition {competition} found at Kaggle.')
        return None
    # 🚀 start a run, with a type to label it and a project it can call home
    wandb.init()
    with wandb.init(project=competition, entity=entity, job_type="download-data") as run:
        path = get_kaggle_dataset(competition)
        sizes = {}
        raw_data = wandb.Artifact(
            competition, type="dataset",
            description=f'Raw {competition} dataset.',
            metadata={"source": "Kaggle"})
        for b, ds, _ in os.walk(path):
            for d in ds:
                files = get_image_files(b+'/'+d)
                sizes[d] = len(files)
                for f in tqdm(files):
                    raw_data.add_reference('file://'+str(f))
        raw_data.metadata['sizes']=sizes
        # ✍️ Save the artifact to W&B.
        run.log_artifact(raw_data)
        return path

In [17]:
#export
def wandb_log(competition, entity=None):
     with wandb.init(project=competition, entity=entity, job_type="download-data") as run:
        path = pathFromCompetition(competition)
        sizes = {}
        raw_data = wandb.Artifact(
            competition, type="dataset",
            description=f'Raw {competition} dataset.',
            metadata={"source": "Kaggle"})
        for b, ds, _ in os.walk(path):
            for d in ds:
                files = get_image_files(b+'/'+d)
                sizes[d] = len(files)
                for f in tqdm(files):
                    raw_data.add_reference('file://'+str(f))
        raw_data.metadata['sizes']=sizes
        # ✍️ Save the artifact to W&B.
        run.log_artifact(raw_data)
        return path

In [13]:
#download_and_log('blah')

In [ ]:
#p = wandb_log('siim-covid19-detection', 'nido')
#p

In [14]:
# export
def setup(competition, entity=None):
    """ Returns Path to local competition files, download and log data if needed.
    """
    # load_dotenv()
    k = KaggleApi()
    k.authenticate()
    try:
        k.competition_list_files(competition)
    except:
        print (f'No competition {competition} found.')
        return None
    try:
        p = pathFromCompetition(competition)
    except:
        if not confirm(f'Download data for {competition} '): return None
        return download_and_log(competition, entity)
    return p

In [15]:
#setup('siim-covid19-detection')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fredguth (use `wandb login --relogin` to force relogin)


$/home/fred/.fastai/archive/siim-covid19-detection.zip does not exist.
Downloading....
$/home/fred/.fastai/data/siim-covid19-detection does not exist.  Extracting...


NameError: name 'tqdm' is not defined

In [ ]:
#setup('dogs-vs-cats-redux-kernels-edition')